In [75]:
import pandas as pd
from reload_recursive import reload_recursive
import os
from loguru import logger
from pathlib import Path
import json
from tqdm.notebook import tqdm
import re
import sys

import mri_data
import monai_training

In [68]:
reload_recursive(mri_data)
reload_recursive(monai_training)

from mri_data.file_manager import DataSet, scan_3Tpioneer_bids
from mri_data import file_manager as fm
from mri_data import utils
from monai_training.preprocess import DataSetProcesser

In [73]:
logger.remove()
logger.add(sys.stderr, level="INFO")

1

In [3]:
drive_root = fm.get_drive_root()
projects_root = Path("/home/srs-9/Projects")
msmri_home = projects_root / "ms_mri"
inference_root = drive_root / "3Tpioneer_bids_predictions"
dataroot = drive_root / "3Tpioneer_bids"
clinical_data_root = drive_root / "Secure_Data"

In [4]:
df = pd.read_excel(clinical_data_root / "manual_labeling_clinicaldata (UNSECURED).xlsx")
df.head()

,ID#,Notes,MRI date,FLAIR contrast?,choroid (label 1),Unnamed: 5,Clinical Notes,pineal (label 2),Unnamed: 8,Dice,...,Unnamed: 19,Unnamed: 20,race,ethnicity,sex,phenotype,symptom_onset,edss,dzdur,age_at_scan.1
0,NaN,NaN,NaT,NaN,Rater 1,Rater 2,NaN,Rater 1,Rater 2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ms1010,NaN,2018-02-08,no,ED,CH,0.72,SRS,CH,0.84,...,NaN,NaN,White,Hispanic or Latino,Female,RRMS,2015-01-18 00:00:00,1.5,3.058096,NaN
2,ms1011,NaN,2018-09-11,yes,ED,CH,0.76,SRS,CH,0.80,...,NaN,NaN,Other,Hispanic or Latino,Female,RRMS,2010-01-01 00:00:00,2,8.692438,NaN
3,ms1019,NaN,2019-06-08,no,ED,CH,0.67,SRS,CH,0.64,...,NaN,NaN,White,Not Hispanic or Latino,Female,RRMS,2009-01-01 00:00:00,3.5,10.430926,NaN
4,ms1033,NaN,2017-01-17,yes,ED,CH,0.75,SRS,CH,0.84,...,NaN,NaN,White,Not Hispanic or Latino,Female,RRMS,1992-01-01 00:00:00,1.5,25.045173,NaN


- Add column which is subid as int
- Set the index to the new column
- Remove any rows where subid is NaN
- Rename columns
- Check "Working with missing data" on pandas documentation

In [31]:
def subject_to_subid(subject):
    if not isinstance(subject, str):
        return None
    re_match = re.match(r"ms(\d{4})", subject)
    if re_match:
        return_val = int(re_match[1])
        return return_val
    else:
        return None

In [44]:
df = pd.read_excel(clinical_data_root / "manual_labeling_clinicaldata (UNSECURED).xlsx")
df = df.convert_dtypes()
keep_columns = ["ID#", "FLAIR contrast?", "phenotype", "age_at_scan", "race", "ethnicity", "sex", "symptom_onset", "edss", "dzdur"]
df = df[keep_columns]
new_columns = {"ID#": "subject", "FLAIR contrast?": "FLAIR contrast", "age_at_scan": "age at scan", "symptom_onset": "symptom onset"}
df.rename(columns=new_columns, inplace=True)
df["subid"] = df['subject'].apply(subject_to_subid)
df.drop(df[df['subid'].isna()].index, inplace=True)
df['subid'] = df['subid'].astype(int)
df = df.set_index("subid")

In [61]:
dataset_proc = DataSetProcesser.new_dataset(dataroot, scan_3Tpioneer_bids, filters=[fm.filter_first_ses])
dataset = dataset_proc.dataset

In [77]:
volumes = dict(pituitary=[], pineal=[], choroid=[])
for subid, _ in tqdm(df.iterrows(), total=len(df)):
    scan = dataset.find_scan(subid=str(subid))[0]
    try:
        pituitary_label = fm.find_label(scan, "pituitary", ["CH", "DT"])
    except FileNotFoundError:
        volumes['pituitary'].append(None)
    else:
        volumes['pituitary'].append(utils.compute_volume(pituitary_label)[1])
    try:
        pineal_label = fm.find_label(scan, "pineal", ["CH", "SRS"])
    except FileNotFoundError:
        volumes['pineal'].append(None)
    else:
        volumes['pineal'].append(utils.compute_volume(pineal_label)[1])
    try:
        choroid_label = fm.find_label(scan, "choroid_t1_flair", ["CH", "ED"])
    except FileNotFoundError:
        volumes['choroid'].append(None)
    else:
        volumes['choroid'].append(utils.compute_volume(choroid_label)[1])

  0%|          | 0/36 [00:00<?, ?it/s]

241025-02:38:51,817 nipype.interface INFO:
	 stdout 2024-10-25T02:38:51.817133:1.000000 2207 1129.992188 
241025-02:38:52,190 nipype.interface INFO:
	 stdout 2024-10-25T02:38:52.190871:1.000000 293 150.017090 
241025-02:38:52,516 nipype.interface INFO:
	 stdout 2024-10-25T02:38:52.516406:1.000000 3776 1933.325928 
241025-02:38:52,841 nipype.interface INFO:
	 stdout 2024-10-25T02:38:52.841358:1.000000 1967 1007.043396 
241025-02:38:53,168 nipype.interface INFO:
	 stdout 2024-10-25T02:38:53.168443:1.000000 554 283.630951 
241025-02:38:53,505 nipype.interface INFO:
	 stdout 2024-10-25T02:38:53.505194:1.000000 4499 2303.349365 
241025-02:38:53,898 nipype.interface INFO:
	 stdout 2024-10-25T02:38:53.898909:1.000000 1087 556.542542 
241025-02:38:54,267 nipype.interface INFO:
	 stdout 2024-10-25T02:38:54.267574:1.000000 260 133.119675 
241025-02:38:54,631 nipype.interface INFO:
	 stdout 2024-10-25T02:38:54.631419:1.000000 6549 3353.079346 
241025-02:38:54,971 nipype.interface INFO:
	 stdout 2

FileNotFoundError: Could not find label matching pituitary for subject 1152 ses 20170529